обучить рукописную сверточную нейронную сеть (с падением размера ядра свертки и последовательностью блоков свертка-пулинг (conv-pool)-(conv-pool)-...) на датасете cifar-10
оценить рост точности при увеличении ширины сети (больше фильтров)
оценить рост точности при увеличении глубины сети (больше слоев)

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Функция, которая создает модель
def create_model(conv_layers=[(4,5,2,2),(8,3,2,2)], 
                               dense_layers=[50], dropout=0.25):
    print('model- conv_layers:', conv_layers, 'dense_layers:',dense_layers)
    model = Sequential()
    # Добавление слоя свертки
    conv_layers_count = 0
    for (filters, kernel_size, pooling_size, pool_stride) in conv_layers:
        conv_layers_count += 1
        if 1 == conv_layers_count:
            # Параметры для первого слоя свертки
            model.add(Conv2D(filters, 
                           (kernel_size, kernel_size), 
                           strides=(1, 1), 
                           activation='relu',
                           padding='same', 
                           input_shape=(32, 32, 3)))
            model.add(Conv2D(filters, 
                           (kernel_size, kernel_size), 
                           strides=(1, 1), 
                           activation='relu',
                           padding='same', 
                           input_shape=(32, 32, 3)))
            model.add(MaxPooling2D(pool_size=(pooling_size, pooling_size), 
                                 strides=(pool_stride, pool_stride))) #сразу следом добавляем слой пулинга
            model.add(Dropout(dropout))
        else:
            # Создание последующих слоев свертки
            model.add(Conv2D(filters, 
                          (kernel_size, kernel_size), 
                          strides=(1, 1),
                          padding='same',
                          activation='relu'))
            model.add(Conv2D(filters, 
                          (kernel_size, kernel_size), 
                          strides=(1, 1),
                          padding='same',
                          activation='relu'))
            model.add(MaxPooling2D(pool_size=(pooling_size, pooling_size), 
                                strides=(pool_stride, pool_stride)))
            model.add(Dropout(dropout))
    # Добавление слоя Flatten полносвязной сети
    model.add(Flatten())
    # Добавление заданного количество скрытых слоев c заданным количеством нейронов
    for nodes in dense_layers:
        model.add(Dense(nodes, activation='relu'))
    # Добавление выходного слоя
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='adam', 
                loss='categorical_crossentropy',  
                metrics=['accuracy'])
    model.summary()
    return model

# создаем экземпляр модели для использования с Grid Search
model = KerasClassifier(build_fn=create_model, verbose=2)

# определяем параметры, которые будут оптимизироваться - добавлены параметры, определяющие конфигурации сверточных слоев и количество и ширину скрытых слоев полносвязной сети
hyperparameters = {
      'conv_layers': [[(4,5,2,2),(8,3,2,2)],
                      [(8,5,2,2),(16,3,2,2)],
                      [(32,5,2,2),(64,3,2,2)]],
      'dense_layers': [[50],[50,20]],
      'epochs':[3, 5, 10],
      'batch_size':[64, 128, 256],
      'dropout':[0.25,0.5]
    }

# запускаем Grid Search
grid = GridSearchCV(estimator=model, param_grid=hyperparameters, n_jobs=-1, cv=3,verbose=2)
grid_result = grid.fit(x_train, y_train)

# выводим результаты
print("Лучший результат: %f с использованием %s" % (grid_result.best_score_, grid_result.best_params_))
# Сетка работала 2 часа
# Лучший результат: 0.721860 с использованием {'batch_size': 128, 'conv_layers': [(32, 5, 2, 2), (64, 3, 2, 2)], 'dense_layers': [50], 'dropout': 0.25, 'epochs': 10}

In [ ]:
# проверка работы обученной модели с оптимальными параметрами
best_model = create_model([(32, 5, 2, 2), (64, 3, 2, 2)],[50],0.25)
best_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001,amsgrad=True),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])


best_model.fit(x_train, y_train, batch_size=128, epochs=20,verbose=1) #увеличение кол-ва эпох до 20 дает прирост метрики ~ на 0,05

scores = best_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# Test loss: 0.6729810237884521
# Test accuracy: 0.7777000069618225

In [ ]:
# проверка работы обученной модели с большим размеров фильтров - пытаемся добиться еще более лучших результатов
best_model = create_model([(64, 5, 2, 2), (128, 3, 2, 2)],[50],0.25)
best_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001,amsgrad=True),
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])


best_model.fit(x_train, y_train, batch_size=128, epochs=20,verbose=1)

scores = best_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# Test loss: 0.8028334379196167
# Test accuracy: 0.7731000185012817

    В итоге получаем наилучшие результаты при параметрах {'batch_size': 128, 'conv_layers': [(32, 5, 2, 2), (64, 3, 2, 2)], 'dense_layers': [50], 'dropout': 0.25, 'epochs': 10}
    Увеличение размера фильтров, количества слоев в полносвязной сети больше вышезаданных не привело к улучшению результатов на тестовой выборке;
    Умеренное увеличение количества эпох, и умеренно малый размер батча приводят к улучшению результатов;
    Структура сети AlexNet оказалась (свертка-свертка-пулинг-свертка-свертка-пулинг) оказалась оптимальной